## Single Inventory on hand and available check

### Define functions

In [1]:
import pandas as pd

# check if string starts with IF
def starts_with_IF(string):
    if string.startswith('IF'):
        return True
    else:
        return False
# check if string starts with IR
def starts_with_IR(string):
    if string.startswith('IR'):
        return True
    else:
        return False
# check if string starts with IT
def starts_with_IT(string):
    if string.startswith('IT'):
        return True
    else:
        return False

# check if string starts with IA
def starts_with_IA(string):
    if string.startswith('IA'):
        return True
    else:
        return False

# check if string starts with CM
def starts_with_CM(string):
    if string.startswith('CM'):
        return True
    else:
        return False
# check if string starts with SO
def starts_with_SO(string):
    if string.startswith('SO'):
        return True
    else:
        return False
# Reset negative value to zero    
def set_zero_if_negative(number):
    if number < 0:
        return 0
    else:
        return number
    




In [2]:
# Check if the string ends with any of the strings in suffix_list
def ends_with_any(string, suffix_list):
    for suffix in suffix_list:
        if string.endswith(suffix):
            return True
    return False



# Check if the string start with Sales
def starts_with_Sales(string):
    if string.startswith('Sales'):
        return True
    else:
        return False


In [3]:
#add value if current id is in the array, append id with its value if it doesn't
def add_value(array, id, value_to_add):
    for i, (current_id, current_value) in enumerate(array):
        if current_id == id:
            array[i] = (current_id, current_value + value_to_add)
            break
    else:  # This else belongs to the for loop, not the if statement
        array.append((id, value_to_add))


In [4]:
# update Sales order value if more than the order value were shipped by IF that were not captured by the Sales Order
def update_SO_Value(array, id, new_value):
    for i, (current_id, current_value) in enumerate(array):
        if current_id == id:
            if current_value < 0: #sales order quantity count in negative
                updated_value = current_value+new_value #update the value by adding the amount shipped
                array[i] = (current_id, updated_value)  #update to the array
                return updated_value if updated_value > 0 else 0 
                #if the amount is greater zero means shipment greater than SO, then return the excess shipped amount. else the shipment is within the order amount return, SO captured already
            else: # when current value is not negative, any additional shipment needs to be record and is effecting the inventory level that were not captured by Sales order
                updated_value = current_value + new_value
                array[i] = (current_id, updated_value)
                return new_value #amount of excess shipment
            break 

# On hand Inventory level function (single item check)

In [5]:
# Calculate the Inventory level (on hand) at each transaction
def single_on_hand(df_test):
    running_sum=0
    for i in range(1,len(df_test)+1):
        if(df_test.loc[df_test.index[-i],'Location'] in ["Seller Warehouse"]): # If the location is Seller Warehouse
            if(starts_with_IR(df_test.loc[df_test.index[-i],'Document Number'])): # If transaction type is "Inventory Receipt"
                if (df_test.loc[df_test.index[-i],'Restock'] == 'Yes'): # If the restock is Yes
                    running_sum += df_test.loc[df_test.index[-i],'Item Count'] # add the item count to the total on hand level
                    onhand_lvl.append(running_sum)
                else:               
                    onhand_lvl.append(running_sum) # level remains the same
            else:
                onhand_lvl.append(running_sum) # level remains the same
        elif (
            starts_with_IF(df_test.loc[df_test.index[-i],'Document Number'])): # If transaction type is "Inventory Fullfillment"
            if(
                df_test.loc[df_test.index[-i],'Status'] in ["Shipped"]): # status is  shipped
                running_sum+=df_test.loc[df_test.index[-i],'Item Count'] # add the item count to the total on hand level
                onhand_lvl.append(running_sum)
            else:
                onhand_lvl.append(running_sum) # level remains the same

        elif(
            starts_with_IR(df_test.loc[df_test.index[-i],'Document Number'])):  # If transaction type is "Inventory Receipt"
            if(df_test.loc[df_test.index[-i],'Restock'] == 'No'): # If the restock is No
                onhand_lvl.append(running_sum) # level remains the same
            else:
                running_sum+=df_test.loc[df_test.index[-i],'Item Count'] # Otherwise,add the item count to the total on hand level
                onhand_lvl.append(running_sum)
        elif(
            starts_with_IT(df_test.loc[df_test.index[-i],'Document Number'])): # If transaction type is "Inventory Transfer"
            running_sum-=df_test.loc[df_test.index[-i],'Item Count'] # Here is only for one by one checking no need to count in real
            onhand_lvl.append(running_sum)
        elif (
                starts_with_SO(df_test.loc[df_test.index[-i],'Document Number'])): # If transaction type is "Sales order"
                onhand_lvl.append(running_sum)
        else:
            running_sum+=df_test.loc[df_test.index[-i],'Item Count'] # All other conditions,add the item count to the total on hand level
            onhand_lvl.append(running_sum)
#     running_sum=set_zero_if_negative(running_sum) # ignore negative inventory level for error checking purpose
    return onhand_lvl



# Available Inventory level function (single item check)

In [6]:
suffix=["QC", "QU","FDA Hold","Hold","Returns"] #locations that has special counting method

# Calculate the Inventory available at each transaction
def single_ava_check(df_test):
    # Initializing values for each parameter, OTHER was to check if the dataframe contains parameter other than IR,IF,IT,IA,SO.
    IR=0
    IF=0
    IT=0
    IA=0
    SO=0
    OTHER=0
    Closed_Cases=[] # Create array to record closed cases for Sales order
    SO_check=[] # Create array to check Sales order
    Cal_Entry=pd.DataFrame(columns=df_test.columns)  # create dataframe to record counted transaction and matches format of the loaded data df_test
    Ncount_Entry=pd.DataFrame(columns=df_test.columns) # create dataframe to record uncounted transaction and matches format of the loaded data df_test
    running_sum=0  # Initializing the available level to zero
    for i in range(1,len(df_test)+1): 
        # If location is in Seller Warehouse
        if(df_test.loc[df_test.index[-i],'Location'] in ["Seller Warehouse"]): 
            if(starts_with_IR(df_test.loc[df_test.index[-i],'Document Number'])):  # If transaction type is "Inventory Receipt"
                if (df_test.loc[df_test.index[-i],'Restock'] == 'Yes'): # If Restock is yes
                    running_sum += df_test.loc[df_test.index[-i],'Item Count'] # add the item count to the total on hand level
                    Inventory_lvl.append(running_sum) #Updated Inventory Level
                    IR+= df_test.loc[df_test.index[-i],'Item Count'] # Count to IR                   
                    Cal_Entry=pd.concat([Cal_Entry,df_test.iloc[[-i]]],ignore_index=True) # Counted transaction
                else:
                    Inventory_lvl.append(running_sum) #Updated Inventory Level which remains the same to the previous one
                    Ncount_Entry=pd.concat([Ncount_Entry,df_test.iloc[[-i]]],ignore_index=True) # Not counted Transaction
            else:
                Inventory_lvl.append(running_sum) #Updated Inventory Level which remains the same to the previous one
                Ncount_Entry=pd.concat([Ncount_Entry,df_test.iloc[[-i]]],ignore_index=True) # Not counted Transaction
        # If transaction type is "Sales order"
        elif (
                starts_with_SO(df_test.loc[df_test.index[-i],'Document Number'])):
            add_value(SO_check,df_test.loc[df_test.index[-i],'Document Number'],df_test.loc[df_test.index[-i],'Item Count'])# Add to So_check
            if(df_test.loc[df_test.index[-i],'Status'] in 
               ["Cancelled","Pending Approval"]): #If status is cancel or pending Approval
                Inventory_lvl.append(running_sum) #Updated Inventory Level which remains the same to the previous one
                Ncount_Entry=pd.concat([Ncount_Entry,df_test.iloc[[-i]]],ignore_index=True) # Not counted Transaction
            elif(df_test.loc[df_test.index[-i],'Status'] in 
               ["Pending Fulfillment"]): #If status is cancel or pending fulfillment
                running_sum+=df_test.loc[df_test.index[-i],'Item Count']  # add the item count to the total on hand level
                Inventory_lvl.append(running_sum) #Updated Inventory Level 
                SO+=df_test.loc[df_test.index[-i],'Item Count'] #Count to SO
                Cal_Entry=pd.concat([Cal_Entry,df_test.iloc[[-i]]],ignore_index=True) # Counted transaction
            elif(df_test.loc[df_test.index[-i],'Status'] == "Closed"): #If status is closed
                Closed_Cases.append(df_test.loc[df_test.index[-i],'Document Number']) # Add to closed cases
                Inventory_lvl.append(running_sum) #Updated Inventory Level which remains the same to the previous one
                Ncount_Entry=pd.concat([Ncount_Entry,df_test.iloc[[-i]]],ignore_index=True) # Not counted Transaction
            else:
                running_sum+=df_test.loc[df_test.index[-i],'Item Count'] # All other conditions,add the item count to the total on hand level
                Inventory_lvl.append(running_sum) #Updated Inventory Level 
                SO+=df_test.loc[df_test.index[-i],'Item Count'] #Count to SO
                Cal_Entry=pd.concat([Cal_Entry,df_test.iloc[[-i]]],ignore_index=True) # Counted transaction
        # If transaction type is "Inventory Fullfillment"
        elif (
            starts_with_IF(df_test.loc[df_test.index[-i],'Document Number'])): 
            if(starts_with_Sales(df_test.loc[df_test.index[-i],'Created From'])):
                #Extrac the created from sales order
                part_after_sales_order = (df_test.loc[df_test.index[-i],'Created From']).split("Sales Order #")[1].strip()
                #Count if the sales order is not in inventory detail (i.e. df_test)
                if(part_after_sales_order not in (id for id, _ in SO_check)):
                    running_sum += df_test.loc[df_test.index[-i],'Item Count'] # add the item count to the total on hand level
                    Inventory_lvl.append(running_sum)
                    IF+= df_test.loc[df_test.index[-i],'Item Count'] 
                    Cal_Entry=pd.concat([Cal_Entry,df_test.iloc[[-i]]],ignore_index=True)
                else:
                    #If Sales order closed and is in IF then counted
                    if(part_after_sales_order in Closed_Cases and df_test.loc[df_test.index[-i],'Status']== "Shipped"):
                        running_sum += df_test.loc[df_test.index[-i],'Item Count'] # add the item count to the total on hand level
                        Inventory_lvl.append(running_sum)
                        IF+= df_test.loc[df_test.index[-i],'Item Count']
                        Cal_Entry=pd.concat([Cal_Entry,df_test.iloc[[-i]]],ignore_index=True)
                    else:
                        #Update the count if excess amount were shipped
                        result=update_SO_Value(SO_check,part_after_sales_order,df_test.loc[df_test.index[-i],'Quantity'])
                        if (result >0):
                            running_sum-=result
                            Inventory_lvl.append(running_sum)
                            IF-=result
                            Cal_Entry=pd.concat([Cal_Entry,df_test.iloc[[-i]]],ignore_index=True)
                        else:
                            Inventory_lvl.append(running_sum)
                            Ncount_Entry=pd.concat([Ncount_Entry,df_test.iloc[[-i]]],ignore_index=True)
            # If it is not created from sales order also need to add to count
            else: 
                running_sum += df_test.loc[df_test.index[-i],'Item Count'] 
                Inventory_lvl.append(running_sum)
                IF+= df_test.loc[df_test.index[-i],'Item Count']
                Cal_Entry=pd.concat([Cal_Entry,df_test.iloc[[-i]]],ignore_index=True)
        # If transaction type is "Inventory Receipt"
        elif(
             starts_with_IR(df_test.loc[df_test.index[-i],'Document Number'])):  
            if( 
                df_test.loc[df_test.index[-i],'Restock'] == 'No' 
                ):
                    running_sum=running_sum #The on hand lvl remains the same
                    Inventory_lvl.append(running_sum)
                    Ncount_Entry=pd.concat([Ncount_Entry,df_test.iloc[[-i]]],ignore_index=True)
            elif(ends_with_any(df_test.loc[df_test.index[-i],'Location'],suffix)):
                    running_sum=running_sum #The on hand lvl remains the same
                    Inventory_lvl.append(running_sum)
                    Ncount_Entry=pd.concat([Ncount_Entry,df_test.iloc[[-i]]],ignore_index=True)
            else:
                running_sum+=df_test.loc[df_test.index[-i],'Item Count'] # Otherwise,add the item count to the total on hand level
                Inventory_lvl.append(running_sum)
                IR+=df_test.loc[df_test.index[-i],'Item Count']
                Cal_Entry=pd.concat([Cal_Entry,df_test.iloc[[-i]]],ignore_index=True)
        # If transaction type is "Inventory Transfer"
        elif(
            starts_with_IT(df_test.loc[df_test.index[-i],'Document Number'])): 
            # IT count negatively 
            if(ends_with_any(df_test.loc[df_test.index[-i],'Location'],suffix)):
                running_sum-=df_test.loc[df_test.index[-i],'Item Count'] # All other conditions,add the item count to the total on hand level
                Inventory_lvl.append(running_sum)
                IT-=df_test.loc[df_test.index[-i],'Item Count']
                Cal_Entry=pd.concat([Cal_Entry,df_test.iloc[[-i]]],ignore_index=True)
            else:
                Inventory_lvl.append(running_sum)
                Ncount_Entry=pd.concat([Ncount_Entry,df_test.iloc[[-i]]],ignore_index=True)
        # If transaction type is "Inventory Ajustment"
        elif(
            starts_with_IA(df_test.loc[df_test.index[-i],'Document Number'])):
            # Not to count when the defined suffix location occurs
            if(ends_with_any(df_test.loc[df_test.index[-i],'Location'],suffix)):
                running_sum=running_sum
                Inventory_lvl.append(running_sum)
            else:
                running_sum+=df_test.loc[df_test.index[-i],'Item Count'] # Otherwise,add the item count to the total on hand level
                Inventory_lvl.append(running_sum)
                IA+=df_test.loc[df_test.index[-i],'Item Count']
                Cal_Entry=pd.concat([Cal_Entry,df_test.iloc[[-i]]],ignore_index=True)
        else:
            running_sum+=df_test.loc[df_test.index[-i],'Item Count'] # All other conditions,add the item count to the total on hand level
            Inventory_lvl.append(running_sum)
            OTHER+=df_test.loc[df_test.index[-i],'Item Count']
            Cal_Entry=pd.concat([Cal_Entry,df_test.iloc[[-i]]],ignore_index=True)
#     running_sum=set_zero_if_negative(running_sum) # ignore negative counts
    return Inventory_lvl,IR,IT,IF,SO,IA,OTHER,Cal_Entry,Ncount_Entry



## Load and filter the data

In [7]:
df=pd.read_csv("531all.csv",low_memory=False)
onhand_lvl=[]
Inventory_lvl=[]
df_test1=df[df['Document Number'].fillna('').str.startswith(('IF','IR','IA',"SO","IT"))]
# # Single Inventory on hand check
## make sure the date is descending order, might need to modify the date format
# df_test1['Date Created']=pd.to_datetime(df_test1['Date Created'])
# df_test1=df_test1.sort_values(by='Date Created',ascending=False)

# Enter the Single Item number to check 

In [8]:
Itemcheck='41121-110-10560-11806'
single_on_hand(df_test1[df_test1['Item']==Itemcheck])
df_test_single=df_test1[df_test1['Item']==Itemcheck]


In [9]:
df_test_single.insert(13,'Onhand_lvl',onhand_lvl[::-1]) # Insert the reverse of the list

In [10]:
KEY=single_ava_check(df_test1[df_test1['Item']==Itemcheck])
df_test_single.insert(15,'Available_lvl',Inventory_lvl[::-1]) # Insert the reverse of the list
df_test_single

,Internal ID,Number,Item,Document Number,Date Created,Status,Item Program,Back Ordered,Location,Bin Number,Restock,Quantity,Item Count,Onhand_lvl,On Hand,Available_lvl,Available,Expiration Date,Created From
559,9917144,YC/DB20230418,41121-110-10560-11806,SO-GWI-135135,5/30/2024 7:07 am,Pending Fulfillment,Regular SD Program,NaN,GWI-New-Jersey,NaN,NaN,500.0,-500.0,24300.0,24300.0,23600.0,23600.0,4/17/2025,NaN
1092,9903599,YC/DB20230418,41121-110-10560-11806,IF127948,5/29/2024 6:18 am,Shipped,Regular SD Program,NaN,GWI-New-Jersey,E-100,NaN,1000.0,-1000.0,24300.0,24300.0,24100.0,23600.0,4/17/2025,Sales Order #SO-GWI-134908
1266,9898414,YC/DB20230914,41121-110-10560-11806,IF127908,5/28/2024 2:25 pm,Shipped,Regular SD Program,NaN,GWI-Chino,RL-40-E,NaN,20.0,-20.0,25300.0,24300.0,24100.0,23600.0,9/13/2025,Sales Order #SO-GWI-134947
1276,9898404,YC/DB20230914,41121-110-10560-11806,SO-GWI-134947,5/28/2024 1:56 pm,Billed,Regular SD Program,NaN,GWI-Chino,NaN,NaN,20.0,-20.0,25320.0,24300.0,24100.0,23600.0,9/13/2025,NaN
1375,9895698,YC/DB20230418,41121-110-10560-11806,SO-GWI-134908,5/28/2024 11:49 am,Billed,Regular SD Program,NaN,GWI-New-Jersey,NaN,NaN,1000.0,-1000.0,25320.0,24300.0,24120.0,23600.0,4/17/2025,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321052,1865793,YC20201109,41121-110-10560-11806,IR43154,2/12/2021 6:23 am,NaN,NaN,NaN,GWI-New-Jersey,N-C-22-A,NaN,500.0,500.0,2500.0,24300.0,2460.0,23600.0,11/8/2022,Purchase Order #PO-GWI-40999
321053,1865793,YC20201109,41121-110-10560-11806,IR43154,2/12/2021 6:23 am,NaN,NaN,NaN,GWI-New-Jersey,N-H-19-A,NaN,1000.0,1000.0,2000.0,24300.0,1960.0,23600.0,11/8/2022,Purchase Order #PO-GWI-40999
321054,1865793,YC20201109,41121-110-10560-11806,IR43154,2/12/2021 6:23 am,NaN,NaN,NaN,GWI-New-Jersey,N-H-08-A,NaN,500.0,500.0,1000.0,24300.0,960.0,23600.0,11/8/2022,Purchase Order #PO-GWI-40999
321055,1865793,YC20201109,41121-110-10560-11806,IR43154,2/12/2021 6:23 am,NaN,NaN,NaN,GWI-New-Jersey,N-D-16-A,NaN,500.0,500.0,500.0,24300.0,460.0,23600.0,11/8/2022,Purchase Order #PO-GWI-40999


# Calcualated result breakdown

In [11]:
print(Itemcheck)
print(f"IR={KEY[1]}")
print(f"IT={KEY[2]}")
print(f"IF={KEY[3]}")
print(f"SO={KEY[4]}")
print(f"IA={KEY[5]}")
print(f"other={KEY[6]} (should be 0 when all other conditions considered)")
print(f"Calculated available={sum(KEY[1:6])}")
print(f"NetSuite available={df_test_single['Available'].iloc[1]}")

41121-110-10560-11806
IR=355160.0
IT=1680.0
IF=-4600.0
SO=-328640.0
IA=0.0
other=0 (should be 0 when all other conditions considered)
Calculated available=23600.0
NetSuite available=23600.0


# Error Checking

In [12]:
# Transactions that are not counted in the calculation
Calculated_rows=pd.DataFrame(KEY[7])
# Calculated_rows[Calculated_rows['Document Number']=='IA6043']
# Calculated_rows.to_csv("Calculated_rows520.csv",index=False)

In [13]:
# Transactions that are counted in the calculation
noCalculated_rows=pd.DataFrame(KEY[8])
# noCalculated_rows[noCalculated_rows['Document Number']=='IA6043']

In [14]:
# Check calculated transaction and calculate the aggregate sum for each unique transaction
Breakdownrows=Calculated_rows[Calculated_rows['Document Number'].str.startswith('SO')]
Breakdownrows.groupby('Document Number').agg(Quantity_Sum=('Item Count', 'sum'))

,Quantity_Sum
Document Number,
SO-GWI-100081,-20.0
SO-GWI-100428,-100.0
SO-GWI-100753,-18440.0
SO-GWI-100754,-7100.0
SO-GWI-100755,-18000.0
...,...
SO-GWI-97981,-20360.0
SO-GWI-98233,-20.0
SO-GWI-98379,-17980.0


In [15]:
# Check non calculated transaction and calculate the aggregate sum for each unique transaction
Breakdownrows = noCalculated_rows[noCalculated_rows['Document Number'].str.startswith(('SO', 'IF'))]
Breakdownrows.groupby('Document Number').agg(Quantity_Sum=('Item Count', 'sum'))

,Quantity_Sum
Document Number,
IF100586,-40.0
IF101043,-160.0
IF101296,-40.0
IF102043,-20.0
IF102367,-1500.0
...,...
IF96482,-18000.0
IF97684,-18000.0
IF98618,-3000.0


# Check inventory calculation with Tableau

In [16]:
py_cal=Calculated_rows
tableau_cal=pd.read_excel("41121.xlsx") # Read the file
tableau_cal=tableau_cal[tableau_cal['Tran Id (Datamart Inventory)'].notna()] # filter out the NA

In [17]:
tableau_cal= tableau_cal.iloc[:,[1,-1]]
tableau_cal.columns=['Document Number','Item Count tableau']
tableau_cal

,Document Number,Item Count tableau
0,IA8095,0.0
1,IA8449,0.0
2,IA8465,20.0
3,IA8512,-20.0
4,IT882,-13000.0
...,...,...
299,SO-GWI-133777,-300.0
300,SO-GWI-134908,-1000.0
301,SO-GWI-134947,-20.0
302,SO-GWI-135135,-500.0


In [18]:
py=py_cal.iloc[:,[3,-5]]
py=py_cal.groupby('Document Number').agg(Quantity_Sum=('Item Count', 'sum'))
py

,Quantity_Sum
Document Number,
IA8095,0.0
IA8449,0.0
IA8465,20.0
IA8512,-20.0
IF121915,-100.0
...,...
SO-GWI-97981,-20360.0
SO-GWI-98233,-20.0
SO-GWI-98379,-17980.0


In [19]:
merge_so=pd.merge(py,tableau_cal,on='Document Number',how='outer')
merge_so['Quantity_Sum']=merge_so['Quantity_Sum'].fillna(0)
merge_so

,Document Number,Quantity_Sum,Item Count tableau
0,IA8095,0.0,0.0
1,IA8449,0.0,0.0
2,IA8465,20.0,20.0
3,IA8512,-20.0,-20.0
4,IF121915,-100.0,0.0
...,...,...,...
299,SO-GWI-111020,0.0,0.0
300,SO-GWI-118144,0.0,0.0
301,SO-GWI-118770,0.0,0.0
302,SO-GWI-121635,0.0,0.0


In [20]:
merge_so['Check']= merge_so['Quantity_Sum'] == merge_so['Item Count tableau']
merge_so

,Document Number,Quantity_Sum,Item Count tableau,Check
0,IA8095,0.0,0.0,True
1,IA8449,0.0,0.0,True
2,IA8465,20.0,20.0,True
3,IA8512,-20.0,-20.0,True
4,IF121915,-100.0,0.0,False
...,...,...,...,...
299,SO-GWI-111020,0.0,0.0,True
300,SO-GWI-118144,0.0,0.0,True
301,SO-GWI-118770,0.0,0.0,True
302,SO-GWI-121635,0.0,0.0,True


In [21]:
k_discrepancy=merge_so[merge_so['Check']==False]
k_discrepancy
# k_discrepancy.to_excel('xxxxxx.xlsx', index=False, engine='openpyxl')

,Document Number,Quantity_Sum,Item Count tableau,Check
4,IF121915,-100.0,0.0,False
5,IF47331,-4500.0,0.0,False
12,IR112603,3520.0,3520.0,False
15,IR43154,12000.0,12000.0,False
31,IT1086,-1360.0,1360.0,False
32,IT1107,20.0,-20.0,False
33,IT1433,-500.0,500.0,False
34,IT1595,580.0,-580.0,False
35,IT1596,-20.0,20.0,False
36,IT1632,-440.0,440.0,False


In [22]:
# Check each type of transaction amount from tableau
tab_SO=tableau_cal[tableau_cal['Document Number'].str.startswith('SO')]
tab_SO=tab_SO.agg(Quantity_Sum=('Item Count tableau', 'sum'))
tab_IR=tableau_cal[tableau_cal['Document Number'].str.startswith('IR')]
tab_IR=tab_IR.agg(Quantity_Sum=('Item Count tableau', 'sum'))
tab_IA=tableau_cal[tableau_cal['Document Number'].str.startswith('IA')]
tab_IA=tab_IA.agg(Quantity_Sum=('Item Count tableau', 'sum'))
tab_IF=tableau_cal[tableau_cal['Document Number'].str.startswith('IF')]
tab_IF=tab_IF.agg(Quantity_Sum=('Item Count tableau', 'sum'))


In [23]:
# Display all dataset
pd.set_option('display.max_rows', None)